# Module 5 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

## Solving Normal Form Games

In [1]:
from typing import List, Tuple, Dict, Callable
from copy import deepcopy

In the lecture we talked about the Prisoner's Dilemma game, shown here in Normal Form:

Player 1 / Player 2  | Defect | Cooperate
------------- | ------------- | -------------
Defect  | -5, -5 | -1, -10
Cooperate  | -10, -1 | -2, -2

where the payoff to Player 1 is the left number and the payoff to Player 2 is the right number. We can represent each payoff cell as a Tuple: `(-5, -5)`, for example. We can represent each row as a List of Tuples: `[(-5, -5), (-1, -10)]` would be the first row and the entire table as a List of Lists:

In [2]:
prisoners_dilemma = [
 [( -5, -5), (-1,-10)],
 [(-10, -1), (-2, -2)]]

prisoners_dilemma

[[(-5, -5), (-1, -10)], [(-10, -1), (-2, -2)]]

In [3]:
self_check_game = [
 [( 10, 10), (14,12),(14,15)],
 [(12,14), (20,20), (28,15)]
 ,[(15,14), (15,28), (25,25)]]

<a id="get_player_values"></a>
## get_player_values

Based on the player selected, will return values of each strategy for a given player

* **game** List[List[Tuple]]: List of all strategies for both players in a game
* **player** int:0 for player 1 or 2 for player 2

**returns** List[List[int]]: a list of lists for a given player

In [5]:
def get_player_values(game:List[List[Tuple]], player:int)-> List[List[int]]:
    values = []
    for row in game:
        row_list = []
        for item in row:
            row_list.append(item[player])
        values.append(row_list)

    return values


In [6]:
# Check prisoner one values are correct, the first value
prisoner_1 = get_player_values(prisoners_dilemma, 0)
assert prisoner_1 == [[-5, -1], [-10, -2]]

#check prisoner 2 values are correct, the second value
prisoner_2 = get_player_values(prisoners_dilemma, 1)
assert prisoner_2 == [[-5, -10], [-1, -2]]

# check a list the same shape as the game
player_1 = get_player_values(self_check_game, 0)
player_2 = get_player_values(self_check_game, 1)

assert len(player_2) == len(self_check_game)
assert len(player_1) == len(self_check_game)

assert len(player_2[0]) == len(self_check_game[0])
assert len(player_1[0]) == len(self_check_game[0])

<a id="dominated"></a>
## dominated

- Find whether the candidate record is dominated by the compare record.
- Each item in the records is evaulated one at a time.
- Dominated by can be weakly or strongly depending on the weak indicator
- Will return true is the candidated is dominated by compare record, else false

* **candidate** List[int]: list of values for a given strategy
* **compare** List[int]:list of values for a given strategy (different than candidate)
* **weak** bool: indicator is dominance should be weakly or strongly. 

**returns** bool: indicator is dominated by other other strategy or not

In [7]:
def dominated(candidate:List[int], compare:List[int], weak:bool)->bool:
    dominated_cnt = equal_cnt = 0
    if len(candidate) != len(compare):
        return False
    # compare scores
    for i in range(len(candidate)):
        if candidate[i] < compare[i]:
            dominated_cnt +=1
        if candidate[i] == compare[i]:
            equal_cnt +=1
    
    if dominated_cnt == len(candidate):# strongly dominated
        return True
    elif equal_cnt == len(candidate): 
        return False
    # weakly dominated
    elif (dominated_cnt + equal_cnt) == len(candidate) and weak:
        return True
    else:
        return False

In [8]:
# check weakly dominated case returns true when weakind is true
print("Weakly Dominate Example\n",player_2[0], player_2[1])
assert dominated(player_2[0], player_2[1], True) == True
# check weakly dominated case returns false when weakind is false
assert dominated(player_2[0], player_2[1], False) == False

# Check strongly dominate case returns true
print("Strongly Dominate Example\n",player_1[0], player_1[1])
assert dominated(player_1[0], player_1[1], False) == True

# check fails if the opposite way
print("Opposite direction of strongly dominate\n",player_1[1], player_1[0])
assert dominated(player_1[1], player_1[0], False) == False

#check false returned is the two lists are diff sizes
assert dominated([5,6], [8,9,10], False) == False

Weakly Dominate Example
 [10, 12, 15] [14, 20, 15]
Strongly Dominate Example
 [10, 14, 14] [12, 20, 28]
Opposite direction of strongly dominate
 [12, 20, 28] [10, 14, 14]


<a id="player_2_eval"></a>
## player_2_eval

- Player 2 strategies are evaulated column wise. 
- Function loops through the strategies (column wise) and finds if a strategy is dominated or not. 
- Dominated can be weakly or strongly based on the weak indicator 
- Will return the stragety (not the value) if a stragety is dominated 
- Otherwise, will return none

* **player** player List[List[int]]: List of all strategy values for a player
* **strategies** List[int]: List of strategies available by name. No values here
* **weak** bool: indicator is dominance should be weakly or strongly. 


**returns** int: strategy that is dominated

In [9]:
def player_2_eval(player:List[List[int]], strategies:List[int], weak:bool)->int:
    strategies_cnt = len(strategies)

    # loop per strategie to check if dominated
    for col in range(strategies_cnt):
        candidate = [row[col] for row in player]

        # find strategy to compare candidate against 
        for col_2 in range(strategies_cnt):

            if col == col_2: # if same strategy then skip 
                continue
            else:
                compare =  [row[col_2] for row in player]
                dom_ind = dominated(candidate, compare, weak)
                if dom_ind:# if dominated col found, return the col name
                    return strategies[col]
    return None               

In [15]:
#check if dominated strategy is found, then the strategy name is returned
strats = [2,4,5]
v = player_2_eval(player_2, strats, False)
assert v in strats
# check player two strategy 2 (first) is returned 
assert v == 2

#check if no dominated col exists, None is returned
player_equal = [[1,1,1],[1,1,1], [1,1,1]]
v2 = player_2_eval(player_equal, strats, False)
assert v2 == None

# check returns first weakly dominated column
player_2_demo = [[20,20,10],[25,10,2], [30,10,1]]
v3 = player_2_eval(player_2_demo, strats, True)
assert v3 == 4

# check the strongly dominated column is returned
v4 = player_2_eval(player_2_demo, strats, False)
assert v4 == 5

<a id="player_1_eval"></a>
## player_1_eval

- Player 1 strategies are evaulated row wise. 
- Function loops through the strategies (row wise) and finds if a strategy is dominated or not. 
- Dominated can be weakly or strongly based on the weak indicator 
- Will return the stragety (not the value) if a stragety is dominated 
- Otherwise, will return none

* **player** player List[List[int]]: List of all strategy values for a player
* **strategies** List[int]: List of strategies available by name. No values here
* **weak** bool: indicator is dominance should be weakly or strongly. 


**returns** int: strategy that is dominated

In [11]:
def player_1_eval(player:List[List[int]], strategies:List[int], weak:bool)->int:
    strategies_cnt = len(strategies)

    # loop per strategie to check if dominated
    for row in range(strategies_cnt):
        candidate = player[row]

        # find strategy to compare candidate against 
        for row_2 in range(strategies_cnt):

            if row == row_2: # if same strategy then skip 
                continue
            else:
                compare =  player[row_2]
                dom_ind = dominated(candidate, compare, weak)
                if dom_ind:# if dominated col found, return the col name
                    return strategies[row]
    return None

In [16]:
#check if dominated strategy is found, then the strategy name is returned
strats = [2,4,5]
v = player_1_eval(player_1, strats, False)
assert v in strats
# check player two strategy 14 (first) is returned 
assert v == 2

#check if no dominated col exists, None is returned
player_equal = [[1,1,1],[1,1,1], [1,1,1]]
v2 = player_1_eval(player_equal, strats, False)
assert v2 == None

# check returns first weakly dominated column
player_1_demo = [[20,20,10],[16,20,2], [15,10,1]]
v3 = player_1_eval(player_1_demo, strats, True)
assert v3 == 4

# check the strongly dominated column is returned
v4 = player_1_eval(player_1_demo, strats, False)
assert v4 == 5

<a id="reduce_board"></a>
## reduce_board

- Reduces the board by removing the weak strategy found
- Returns a reduced board for both players and reduces strategy list

* **player1**  List[List[int]]: List of strategies currently left for player 1. Contains values of strategies
* **player2** List[List[int]]:List of strategies currently left for player 2. Contains values of strategies
* **strategies** List[int]: List of current strategies available for a given player (player field in params)
* **eliminate_strat** int: strategy to eliminate. This is the weak strategy
* **player** int: player for the weak strategy that is going to be remove. 1 or 2 


**returns** Tupe[List[List[int]],List[List[int]],List[int]]: both players remianing strategy values and list of strategies list for a given player

In [13]:
def reduce_board(player1:List[List[int]], player2:List[List[int]], strategies:List[int], eliminate_strat:int, player:int)->Tuple:
    p1 = deepcopy(player1)
    p2 = deepcopy(player2)
    strat_copy = deepcopy(strategies)
    strat_index = strategies.index(eliminate_strat)
    
    if player == 1:# remove by row
        p1.pop(strat_index)
        p2.pop(strat_index)
        strat_copy.pop(strat_index)
    
    elif player ==2 :#remove by col
        p1 = [[row[col] for col in range(len(row)) if col != strat_index ] for row in p1]
        p2 = [[row[col] for col in range(len(row)) if col != strat_index ] for row in p2]
        strat_copy.pop(strat_index)
    
    else:# not a valid player selected
        pass
    return p1, p2, strat_copy

In [14]:
strats_test = [2,4,5]
p1, p2, strat = reduce_board(player_1, player_2,strats_test, 4, 1)
# verfiy none of the original lists changed
assert player_1 != p1
assert player_2 != p2
assert strat != strats_test

#verify the length of each list is one smaller in size
# removal of a row (player 1)
assert len(player_1) -1 == len(p1)
assert len(player_2) -1 == len(p2)
assert len(strat) == len(strats_test) -1

#verify removing by col (player 2)
# each row should be one size smaller
p3, p4, strat2 = reduce_board(p1, p2, [2,4,5], 4, 2)
for row_index in range(len(p3)):
    assert len(p3[row_index]) == len(p1[row_index]) - 1
    assert len(p4[row_index]) == len(p2[row_index]) - 1
assert len(strat2) == len(strats_test) -1

in which case the strategies are represented by indices into the List of Lists. For example, `(Defect, Cooperate)` for the above game becomes `prisoners_dilemma[ 0][ 1]` and returns the payoff `(-1, -10)` because 0 is the first row of the table ("Defect" for Player 1) and 1 is the 2nd column of the row ("Cooperate" for Player 2).

For this assignment, you are going write a function that uses Successive Elimination of Dominated Strategies (SEDS) to find the **pure strategy** Nash Equilibrium of a Normal Form Game. The function is called `solve_game`:

```python
def solve_game( game: List[List[Tuple]], weak=False) -> Tuple:
    pass # returns strategy indices of Nash equilibrium or None.
```

and it takes two parameters: the game, in a format that we described earlier and an optional boolean flag that controls whether the algorithm considers only **strongly dominated strategies** (the default will be false) or whether it should consider **weakly dominated strategies** as well.

It should work with game matrices of any size and it will return the **strategy indices** of the Nash Equilibrium. If there is no **pure strategy** equilibrium that can be found using SEDS, return `None`.


<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
Do not return the payoff. That's not useful. Return the strategy indices, any other output is incorrect.
</div>

As before, you must provide your implementation in the space below, one Markdown cell for documentation and one Code cell for implementation, one function and assertations per Codecell.


---

<a id="solve_game"></a>
## solve_game

- Using the Successive Elimination of Dominated Strategies (SEDS), returns the **pure strategy** Nash Equilibrium of a Normal Form Game.
- The strategy is returned not the value of pure strategy found

* **game** List[List[Tuple]]: List of all strategies per both players in a game
* **weak** bool: indicator is dominance should be weakly or strongly. 


**returns** Tuple(int,int): returns the Nash Equilibrium

In [17]:
def solve_game(game: List[List[Tuple]], weak:bool=False) -> Tuple:
    strategies_remaining = [[i for i in range(len(game))], [i for i in range(len(game[0]))]] # [player1, player2]
    player_1 = get_player_values(game, 0)
    player_2 = get_player_values(game, 1)
    
    while(True): 
        if len(strategies_remaining[1]) ==len(strategies_remaining[0]) == 1: # only one strategy left for each player
            return (strategies_remaining[0][0], strategies_remaining[1][0])
        player1_eliminate = player_1_eval(player_1, strategies_remaining[0], weak)
        if player1_eliminate != None:
            player1_changes = reduce_board(player_1, player_2, strategies_remaining[0],player1_eliminate, 1)
            player_1, player_2, strategies_remaining[0] = player1_changes

        player2_eliminate = player_2_eval(player_2, strategies_remaining[1], weak)
        if player2_eliminate != None:
            player2_changes = reduce_board(player_1, player_2, strategies_remaining[1],player2_eliminate, 2)
            player_1, player_2, strategies_remaining[1] = player2_changes
        if player2_eliminate == None and player1_eliminate == None: # no weak strat found for either player
            return None         
    return None

In [18]:
prisoners_dilemma
# check game can be solved with multiple shaped games
prison_strat = solve_game(prisoners_dilemma, True)
assert prison_strat != None
# self check 
self_check_strat = solve_game(self_check_game, True)
assert self_check_strat != None

#check only a single tuple is returned
assert len(prison_strat)  == 2

# check solution is index is returned not value of strategy
# all prisoner values are negative
assert prison_strat[0] >= 0 and prison_strat[1] >= 0

#check strategy is correct

assert prison_strat == (0,0)
assert self_check_strat == (1,1)

## Additional Directions

Create three games as described and according to the following:

1. Your games must be created and solved "by hand".
2. The strategy pairs must **not** be on the main diagonal (0, 0), (1, 1), or (2, 2). And the solution cannot be the same for both Game 1 and Game 2.
3. Make sure you fill out the Markdown ("?") with your game as well as the solution ("?").
4. Remember, **do not return the payoff**, return the strategy indices.

For games that can be solved with *weak* SEDS, there may be more than one solution. You only need to return the first solution found. However, if you would like to return all solutions, you can implement `solve_game` as state space search.

### Test Game 1. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Strongly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 5,7 | 7,8 | 7,9 |
|1  | 10,8 | 10,1 | 18,3 |
|2  | 8,6 | 8,2 | 10,4 |

**Solution:** (1,0)

In [19]:
test_game_1 = [
    [(5,7),(7,8),(7,9)]
    ,[(10,8),(10,1),(18,3)]
    ,[(8,6),(8,2),(10,4)]]

solution = solve_game(test_game_1)
print(solution)

(1, 0)


In [21]:
assert solution == (1,0) # insert your solution from above.

### Test Game 2. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Weakly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1,7 | 2,8 | 4,9 |
|1  | 10,4 | 8,2| 10,4 |
|2  | 10,8 | 10,1 | 18,3 |

**Solution:** (2,0)

In [23]:
test_game_2 = [
    [(1,7),(2,8),(4,9)]
    ,[(10,4),(8,2),(10,4)]
    ,[(10,8),(10,1),(18,3)]
    ]

strong_solution = solve_game( test_game_2)
weak_solution = solve_game( test_game_2, weak=True)

In [24]:
assert strong_solution == None
assert weak_solution == (2,0) # insert your solution from above.

### Test Game 3. Create a 3x3 two player game

**that cannot be solved using the Successive Elimintation of Dominated Strategies at all**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 7,3| 2,9 | 3,1 |
|1  | 4,3 | 2,1 | 9,6 |
|2  | 2,5 | 5,6 | 1,8 |

**Solution:** None

In [25]:
test_game_3 = [
    [(7,3),(2,9),(3,1)]
    ,[(4,3),(2,1),(9,6)]
    ,[(2,5),(5,6),(1,8)]]

strong_solution = solve_game( test_game_3)
weak_solution = solve_game( test_game_3, weak=True)

In [26]:
assert strong_solution == None
assert weak_solution == None

## Before You Submit...

1. Did you provide output exactly as requested? **Don't forget to fill out the Markdown tables with your games**.
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.